# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from __future__ import annotations
from random import choices, randint
from dataclasses import dataclass, field
from copy import deepcopy
import numpy as np

import lab9_lib

In [2]:
PROBLEM_SIZE = 10
LOCI = 1000

In [3]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

ind = choices([1], k=50)
print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")
print(fitness.calls)

11110011110000011011111110010010111111000010001111: 23.33%
00011111101011111010101011111110111101001110101101: 9.11%
01000001111111100100001110001111011010010101001111: 23.33%
01000011010001100000111111110011101101010011000000: 29.56%
01110101100010011000000100100110000011111110110100: 23.56%
01111011111001000110110111000111000000100000000111: 7.33%
11000110010011000101101111101011000000010100000010: 17.56%
10010111111101011101000110010101111011011001000110: 19.13%
11110101011001101110000001010111001000011000010001: 15.34%
00110100011101111001111010011100101101010101111001: 9.11%
11111111111111111111111111111111111111111111111111: 100.00%
11


In [4]:
@dataclass(frozen=True, init=False)
class Individual:
    _n_loci: int
    _genotype: tuple[int]

    def __init__(self, n_loci: int, genotype: tuple[int] = None):
        object.__setattr__(self, "_n_loci", n_loci)
        if genotype is None:
            genotype = tuple(choices([0, 1], k=n_loci))
        object.__setattr__(self, "_genotype", genotype)
        assert len(self._genotype) == self._n_loci, 'n_loci field and genotype length do not match'

    @property
    def n_loci(self):
        return self._n_loci

    @property
    def genotype(self):
        return tuple(self._genotype)

    def __str__(self) -> str:
        return f"{''.join(str(gene) for gene in self.genotype)}"

In [5]:
def mutate(ind: Individual, *, eps: int) -> Individual:
    n_loci, genotype = ind.n_loci, list(ind.genotype)
    first_one_index, last_one_index = (
        genotype.index(1) - eps,
        n_loci - genotype[::-1].index(1) + (eps - 1),
    )
    range = np.clip([first_one_index, last_one_index], 0, n_loci - 1).tolist()
    index = randint(*range)
    genotype[index] = 1 - genotype[index]
    return Individual(n_loci, tuple(genotype))

In [6]:
ind = Individual(LOCI)
eps = int(PROBLEM_SIZE * 0.3)
offspring = mutate(ind, eps=eps)

In [15]:
for index, (first, second) in enumerate(zip(offspring.genotype, ind.genotype)):
    if first != second:
        print(index, first, second)

29 1 0
